# Taask 4.2 Demonstrate running Circuits on Real Hardware

## Objective 1 : Execute on Target Hardware

Check Task 4.1 Objective 1

## Objective 2: Primitives

* Abstraction
* Estimator: expectation value of observable
* Sampler: samples the output register

### Implmentations
* EstimatorV2
* SamplerV2
* StateVectorEstimator
* StateVectorSampler
* BackendEstimatorV2
* BackendSamplerV2

### Estimator

Output is in PubResult : expectation value and their standard error

In [ ]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from test_utils import read_account_file
account = read_account_file('account_info.conf')  # Use the updated account file name
service = QiskitRuntimeService(channel="ibm_cloud", instance=account['instance'], token=account['token'])
backend = service.least_busy()
print(f"Using backend: {backend.name}")

# Create a quantum circuit for entanglement and measurement then draw it
qc = QuantumCircuit(2, 2)
qc.h(0)  # Apply Hadamard gate to qubit 0
qc.cx(0, 1)  # Apply CNOT gate to entangle qubit 0 and qubit 1
qc.measure([0, 1], [0, 1])  # Measure both qubits
qc.draw("mpl")
from qiskit.quantum_info import SparsePauliOp
ZZ = SparsePauliOp.from_list([("ZZ", 1)])  # Define the observable for ZZ measurement

pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
# Transpile the circuit
transpiled_circuit = pm.run(qc)

estimator = Estimator(mode=backend)
# map observables to the circuit
observables = [ZZ.apply_layout(transpiled_circuit.layout)]
# run estimator to get the expectation value
job = estimator.run([(transpiled_circuit, observables)])
print(f"Job ID: {job.job_id()}")

Using backend: ibm_torino


### Sampler

Output is weighted samples such as counts

In [3]:
from qiskit_ibm_runtime import SamplerV2 as Sampler

sampler = Sampler(mode=backend)
# Sample the transpiled circuit
job = sampler.run([transpiled_circuit], shots=1024)
print(f"Job ID: {job.job_id()}")

Job ID: d31etp1mc66s738e7sig
